I have already downloaded the required parquet and csv file and have placed them in a directory named "data".

In [1]:
import pandas as pd

zones = pd.read_csv('data/taxi_zone_lookup.csv')

zones.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [2]:

green_taxis = pd.read_parquet('data/green_tripdata_2025-11.parquet')

green_taxis.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.2,...,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.0
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.2,...,0.5,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.0
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.5,...,0.5,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.0
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.7,...,0.5,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.0
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.4,...,1.5,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.0


## Question 3 - Counting short trips
For the trips in November 2025 (`lpep_pickup_datetime` between '2025-11-01' and '2025-12-01', exclusive of the upper bound), how many trips had a trip_distance of less than or equal to 1 mile?

In [3]:
green_taxis.dtypes

VendorID                          int32
lpep_pickup_datetime     datetime64[us]
lpep_dropoff_datetime    datetime64[us]
store_and_fwd_flag                  str
RatecodeID                      float64
PULocationID                      int32
DOLocationID                      int32
passenger_count                 float64
trip_distance                   float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
ehail_fee                       float64
improvement_surcharge           float64
total_amount                    float64
payment_type                    float64
trip_type                       float64
congestion_surcharge            float64
cbd_congestion_fee              float64
dtype: object

From the command above, we see that `lpep_pickup_datetime` and `lpep_dropoff_datetime` are both date types. This means where do row filtering on the `lpep_pickup_datetime`.

In [4]:
pickup_df = green_taxis[
    ( green_taxis["lpep_pickup_datetime"].dt.date >= pd.to_datetime('2025-11-01').date() ) & \
    ( green_taxis["lpep_pickup_datetime"].dt.date < pd.to_datetime('2025-12-01').date() )
]

print("{} trips less than or equal to 1.0 mile".format(pickup_df[ pickup_df["trip_distance"] <= 1.0 ].shape[0]))

8007 trips less than or equal to 1.0 mile


## Question 4 - Longest trip for each day

Which was the pick up day with the longest trip distance? Only consider trips with trip_distance less than 100 miles (to exclude data errors).

Use the pick up time for your calculations.

Since `trip_distance` is type `float64` we can do row filtering to select rows that are strictly less than 100 miles. Afterwards, we can can filter the result set even more by selecting the row with the highest trip distance.

In [5]:
longest_trip = green_taxis[
    green_taxis["trip_distance"] < 100    
]

longest_trip = longest_trip[ longest_trip["trip_distance"] == longest_trip["trip_distance"].max() ].filter(items=["lpep_pickup_datetime", "trip_distance"])

longest_trip

,lpep_pickup_datetime,trip_distance
18867,2025-11-14 15:36:27,88.03


In [6]:
print("The longest pickup day is {} with {} miles".format(longest_trip["lpep_pickup_datetime"].item(), longest_trip["trip_distance"].item()))

The longest pickup day is 2025-11-14 15:36:27 with 88.03 miles


## Question 5 - Biggest pickup zone
Which was the pickup zone with the largest `total_amount` (sum of all trips) on November 18th, 2025?

In [7]:
t = green_taxis[
    green_taxis["lpep_pickup_datetime"].dt.date == pd.to_datetime("2025-11-18").date()
].groupby(["PULocationID"]).agg(total_amount=('total_amount', 'sum')).sort_values(by='total_amount', ascending=False)

result = pd.merge(t, zones, left_on="PULocationID", right_on="LocationID").head(1)

print("The pickup zone with largest 'total_amount' on 2025-11-18 is {}".format(result["Zone"].item()))

The pickup zone with largest 'total_amount' on 2025-11-18 is East Harlem North


## Question 6 -  Largest tip
For the passengers picked up in the zone named "East Harlem North" in November 2025, which was the drop off zone that had the largest tip?

In [8]:
nov_taxis = green_taxis[
    ( green_taxis["lpep_pickup_datetime"].dt.date >= pd.to_datetime("2025-11-01").date() ) & \
    ( green_taxis["lpep_pickup_datetime"].dt.date <  pd.to_datetime("2025-12-01").date() )
]

nov_taxis.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.2,...,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.0
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.2,...,0.5,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.0
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.5,...,0.5,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.0
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.7,...,0.5,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.0
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.4,...,1.5,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.0


In [9]:
nov_taxis = nov_taxis.merge(
    zones, 
    left_on="PULocationID", 
    right_on="LocationID"
) 

nov_taxis.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee,LocationID,Borough,Zone,service_zone
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.2,...,1.0,11.64,1.0,1.0,0.00,0.0,74,Manhattan,East Harlem North,Boro Zone
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.2,...,1.0,9.70,2.0,1.0,0.00,0.0,74,Manhattan,East Harlem North,Boro Zone
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.5,...,1.0,21.00,1.0,1.0,0.00,0.0,83,Queens,Elmhurst/Maspeth,Boro Zone
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.7,...,1.0,27.70,1.0,1.0,0.00,0.0,166,Manhattan,Morningside Heights,Boro Zone
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.4,...,1.0,24.65,1.0,1.0,2.75,0.0,166,Manhattan,Morningside Heights,Boro Zone


In [10]:
tips = nov_taxis[ 
    nov_taxis["Zone"] == "East Harlem North"
].filter(items=["tip_amount", "DOLocationID"])

largest_tip = tips[ 
    tips["tip_amount"] == tips["tip_amount"].max() 
].merge(zones, left_on="DOLocationID", right_on="LocationID")

In [11]:
print(
    """The drop off zone that had the largest tip within the 'East Harlem North' 
pickup zone in November 2025 is {}
    """.format(largest_tip["Zone"].item())
)

The drop off zone that had the largest tip within the 'East Harlem North' 
pickup zone in November 2025 is Yorkville West
    
